In [ ]:
import os, glob
import numpy as np
import random
import io

from PyPDF2 import PdfReader

In [ ]:
pdfReaders = []
pdfFiles = []

In [ ]:
# read files and extract to .txt files
def read_files():
    os.chdir("/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored")
    for file in glob.glob("**/*.pdf"):
        pdfFiles.append(file)
        pdfReaders.append(PdfReader(file))
    print("pdf files read")
        
def extract_to_txt():
    os.chdir("/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/")
    for i in pdfReaders:
        with open(i.metadata.title + ".txt", 'w', encoding="utf-8") as file:
            for j in range(0, len(i.pages)):
                file.write(i.getPage(j).extract_text())
    print("text extraction complete")


In [ ]:
read_files()

In [ ]:
extract_to_txt()